In [1]:
from train import train
from data import Flickr8k
from encoder_decoder import ResNetEncoder, Decoder


In [2]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

In [3]:
args = AttrDict()
# You can play with the hyperparameters here, but to finish the assignment,
# there is no need to tune the hyperparameters here.
args_dict = {
    "learn_rate": 0.05,
    "batch_size": 64,
    "epochs": 10,
}
args.update(args_dict)

In [4]:
train_data = Flickr8k(csv_file="flickr8k/train.csv", root_dir="flickr8k/images")

In [5]:
e = ResNetEncoder(20)
d = Decoder(len(train_data.vocab), 20, 60)

In [6]:
train(e, d, args)

torch.Size([64, 20])
torch.Size([64, 23, 20])
torch.Size([64, 24, 20])
torch.Size([64, 24, 8256])
torch.Size([64, 23])


ValueError: Expected target size (64, 8256), got torch.Size([64, 23])